In [1]:
import pandas as pd

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

##### Fungsi prepare_feature jika temen2 ingin mengolah feature2, seperti menghilangkan feature, transformasi feature, dll

In [3]:
def prepare_feature(train_data,test_data=None):
    train2 = train_data.copy()
    test2 = test_data.copy()
    categ = {}
    for y in train.columns:
        if train2[y].dtype == 'object':
            categ[y] = {k: 0 for i, k in enumerate(train2[y].unique())}
    for colm in categ.keys():
        for k in train[colm].unique():
            if k is not np.nan:
                key = colm+"_"+k
                val = train2[colm]
                val2 = test2[colm]
                train2[key] = [1 if k == j else 0 for j in val]
                test2[key]= [1 if k == j else 0 for j in val2] 
        del train2[colm]
        del test2[colm]
    train2 = train2.fillna(0)
    test2 = test2.fillna(0)
    return train2,test2

In [4]:
def fillna(trn):
    return trn.fillna(0)

In [5]:
def clean_object_column(trn):
    trn2 = trn.copy()
    for y in trn2.columns:
        if trn2[y].dtype == 'object':
            del trn2[y]
    return trn2

#### pada fungsi train_model ini, silahkan memilih algorithm yang menurut temen2 cocok. Sebagai tahap silahkan menggunakan linear regression

In [6]:
def train_model(trainX,trainY):
    model = XGBRegressor()
    model.fit(trainX, trainY)
    print("MSE Training data: ", np.sqrt(mean_squared_error(trainY, model.predict(trainX))))
    print("MEA Training data: ", mean_absolute_error(trainY, model.predict(trainX)))
    return model

In [7]:
def eval(model, feat,label):
    pred = model.predict(feat)
    print("MSE : ", np.sqrt(mean_squared_error(label, pred)))
    print("MEA : ", mean_absolute_error(label, pred))

In [8]:
def transform(data):
    data2 = data.copy()
    data2 = prepare_feature(data2)
    # train2 = clean_object_column(train)
    data2 = fillna(data2)
    print(data2.shape)
    return data2

In [9]:
def predict_result(model, test_data): 
    feature,_ = get_feature_label(test_data)
    submission = pd.DataFrame({"Id": test_data["Id"], "SalePrice": model.predict(feature)})
    submission.to_csv("data/submission.csv", index=False)
    print('saved to submission.csv')
    return submission

In [10]:
def get_feature_label(train_data):
    label = None
    if 'SalePrice' in train_data.columns:
        label = train_data.SalePrice
    lbl = []
    for c in train_data.columns:
        if c != 'SalePrice' and c != 'Id':
            lbl.append(c)
    feat = train_data[lbl]
    return feat,label

In [11]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [12]:
train2,test2 = prepare_feature(train, test)
feat,label = get_feature_label(train2)
trainX,testX,trainY,testY = train_test_split(feat, label)

In [13]:
model = train_model(trainX,trainY)

MSE Training data:  13620.342343558814
MEA Training data:  9942.233632990867


In [14]:
eval(model,testX,testY)

MSE :  44250.932547374694
MEA :  17901.47243150685


In [15]:
final_model = train_model(feat, label)

MSE Training data:  15305.782146551466
MEA Training data:  10941.005730950343


In [17]:
submission = predict_result(final_model, test2)

saved to submission.csv
